# 📊 Food Delivery Analysis in Databricks

In [ ]:

# Load CSV into Spark DataFrame
df = spark.read.csv("/FileStore/tables/24MBMA47_order.csv", header=True, inferSchema=True)

# Show sample rows
df.show(5)

# Print schema
df.printSchema()


In [ ]:

# Register DataFrame as SQL table
df.createOrReplaceTempView("orders")


## 🔍 Analysis Queries

In [ ]:

# Total Order Value per Customer
spark.sql("""
SELECT customer_id, SUM(order_amount) AS total_spent
FROM orders
GROUP BY customer_id
ORDER BY total_spent DESC
""" ).show()


In [ ]:

# Average Delivery Time per Partner
from pyspark.sql.functions import unix_timestamp, col, avg

df2 = df.withColumn("delivery_duration",
                    unix_timestamp("delivery_time") - unix_timestamp("order_time"))

df2.groupBy("delivery_partner_id").agg(avg("delivery_duration").alias("avg_delivery_time")).show()


In [ ]:

# Customers with Frequent Cancellations
spark.sql("""
SELECT customer_id, COUNT(*) AS cancellations
FROM orders
WHERE order_status = 'Cancelled'
GROUP BY customer_id
HAVING COUNT(*) > 1
""" ).show()


In [ ]:

# Peak Order Hours
spark.sql("""
SELECT HOUR(order_time) AS order_hour, COUNT(*) AS total_orders
FROM orders
GROUP BY HOUR(order_time)
ORDER BY total_orders DESC
""" ).show()


## 📈 Visualizations

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd

# Convert to Pandas for visualization
orders_pd = df.toPandas()
orders_pd['order_hour'] = pd.to_datetime(orders_pd['order_time']).dt.hour

# Peak Order Hours
orders_pd['order_hour'].value_counts().sort_index().plot(kind='bar', figsize=(8,5))
plt.xlabel("Hour of Day")
plt.ylabel("Number of Orders")
plt.title("Peak Order Hours")
plt.show()


In [ ]:

# Customer Spending (Top 10)
customer_spending = orders_pd.groupby("customer_id")["order_amount"].sum().sort_values(ascending=False).head(10)

customer_spending.plot(kind='bar', figsize=(8,5))
plt.xlabel("Customer ID")
plt.ylabel("Total Spending")
plt.title("Top 10 Customers by Total Spending")
plt.show()


In [ ]:

# Average Delivery Time per Partner
df2_pd = df2.toPandas()
avg_time = df2_pd.groupby("delivery_partner_id")["delivery_duration"].mean().sort_values()

avg_time.plot(kind='bar', figsize=(8,5))
plt.xlabel("Delivery Partner")
plt.ylabel("Avg Delivery Time (seconds)")
plt.title("Average Delivery Time per Partner")
plt.show()


## 💾 Export Results (Optional)

In [ ]:

# Save customer spending result to DBFS
df.groupBy("customer_id").sum("order_amount") \

  .toPandas().to_csv("/dbfs/FileStore/output/customer_spending.csv", index=False)
